In [19]:
import pandas as pd
import json

In [20]:
with open('openaq_data.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data['results'])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location     10000 non-null  object 
 1   parameter    10000 non-null  object 
 2   value        10000 non-null  float64
 3   date         10000 non-null  object 
 4   unit         10000 non-null  object 
 5   coordinates  10000 non-null  object 
 6   country      10000 non-null  object 
 7   city         0 non-null      object 
dtypes: float64(1), object(7)
memory usage: 625.1+ KB


In [22]:
df.head()

,location,parameter,value,date,unit,coordinates,country,city
0,LANSING,nox,0.0025,"{'utc': '2024-04-25T19:00:00+00:00', 'local': ...",ppm,"{'latitude': 42.76138, 'longitude': -84.562867}",US,None
1,Mooroolbark,pm10,6.5300,"{'utc': '2024-04-25T19:00:00+00:00', 'local': ...",µg/m³,"{'latitude': -37.77512, 'longitude': 145.3284}",AU,None
2,Footscray,pm25,1.2200,"{'utc': '2024-04-25T19:00:00+00:00', 'local': ...",µg/m³,"{'latitude': -37.80266, 'longitude': 144.8778}",AU,None
3,Footscray,pm10,6.1600,"{'utc': '2024-04-25T19:00:00+00:00', 'local': ...",µg/m³,"{'latitude': -37.80266, 'longitude': 144.8778}",AU,None
4,Mooroolbark,pm25,5.4500,"{'utc': '2024-04-25T19:00:00+00:00', 'local': ...",µg/m³,"{'latitude': -37.77512, 'longitude': 145.3284}",AU,None


In [23]:
df = df.drop(columns=['city'])
df = df.rename(columns={'location': 'city'})

In [24]:
df = df[~df['city'].str.contains(r'[A-Z]+[0-9]+')]

In [25]:
df[['latitude', 'longitude']] = df['coordinates'].apply(lambda x: pd.Series([x['latitude'], x['longitude']]))

In [26]:
df.drop(columns=['coordinates'], inplace=True)

In [27]:
df.drop(columns=['date'], inplace=True)

In [28]:
european_country_codes = ['IS', 'GR', 'NL', 'NO', 'RO', 'MK', 'ES', 'DK', 'FI', 'LU',
                       'LV', 'IE', 'BG', 'PL', 'RU', 'SI', 'XK', 'IT', 'RS', 'PT',
                       'HU', 'FR', 'SE', 'DE', 'EE', 'MD', 'AT', 'UA', 'CZ', 'BE', 'SK']


In [29]:
df = df[df['country'].isin(european_country_codes)]

In [30]:
country_names = {
    'IS': 'Iceland', 'GR': 'Greece', 'NL': 'Netherlands', 'NO': 'Norway', 'RO': 'Romania', 
    'MK': 'North Macedonia', 'ES': 'Spain', 'DK': 'Denmark', 'FI': 'Finland', 'LU': 'Luxembourg', 
    'LV': 'Latvia', 'IE': 'Ireland', 'BG': 'Bulgaria', 'PL': 'Poland', 'RU': 'Russia', 
    'SI': 'Slovenia', 'XK': 'Kosovo', 'IT': 'Italy', 'RS': 'Serbia', 'PT': 'Portugal', 
    'HU': 'Hungary', 'FR': 'France', 'SE': 'Sweden', 'DE': 'Germany', 'EE': 'Estonia', 
    'MD': 'Moldova', 'AT': 'Austria', 'UA': 'Ukraine', 'CZ': 'Czech Republic', 
    'BE': 'Belgium', 'SK': 'Slovakia'
}

In [31]:
df['country'] = df['country'].map(country_names)

In [32]:
df.head()

,city,parameter,value,unit,country,latitude,longitude
6,Reykjavik Husdyragardurinn,o3,35.961900,µg/m³,Iceland,64.138861,-21.873411
7,VOLOS-1,pm25,2.000000,µg/m³,Greece,39.366711,22.942922
8,VOLOS-1,pm10,18.000000,µg/m³,Greece,39.366711,22.942922
31,Utrecht NLH,relativehumidity,64.237500,%,Netherlands,52.094751,5.134536
32,Utrecht NLH,pm1,14.333333,µg/m³,Netherlands,52.094751,5.134536


In [33]:
df["parameter"].unique()

array(['o3', 'pm25', 'pm10', 'relativehumidity', 'pm1', 'um003', 'so2',
       'no2', 'co', 'bc', 'temperature', 'no'], dtype=object)

In [34]:
df['parameter'].value_counts()

parameter
no2                 540
pm10                517
pm25                351
o3                  345
so2                 284
pm1                  20
relativehumidity     17
um003                16
temperature          15
co                   13
bc                    2
no                    1
Name: count, dtype: int64

In [35]:
#filter df

In [36]:
filtered_df = df.drop(df[df['parameter'].isin(['pm1', 'relativehumidity', 'um003', 'temperature', 'bc'])].index)

In [37]:
filtered_df['parameter'].value_counts()

parameter
no2     540
pm10    517
pm25    351
o3      345
so2     284
co       13
no        1
Name: count, dtype: int64

In [38]:
#Transformation

In [39]:
def label_parameter(row):
    value = row['value']
    parameter = row['parameter']
    
    if parameter == 'nox':
        if value < 40:
            return 'Good'
        elif value < 80:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'pm25':
        if value < 12:
            return 'Good'
        elif value < 35.4:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'pm10':
        if value < 50:
            return 'Good'
        elif value < 150:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'no2':
        if value < 40:
            return 'Good'
        elif value < 80:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'so2':
        if value < 40:
            return 'Good'
        elif value < 80:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'o3':
        if value < 50:
            return 'Good'
        elif value < 100:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'no':
        if value < 40:
            return 'Good'
        elif value < 100:
            return 'Moderate'
        else:
            return 'Unhealthy'
    elif parameter == 'co':
        if value < 5:
            return 'Good'
        elif value < 10:
            return 'Moderate'
        else:
            return 'Unhealthy'

In [40]:
filtered_df['quality_label'] = filtered_df.apply(label_parameter, axis=1)

In [41]:
filtered_df.head()

,city,parameter,value,unit,country,latitude,longitude,quality_label
6,Reykjavik Husdyragardurinn,o3,35.961900,µg/m³,Iceland,64.138861,-21.873411,Good
7,VOLOS-1,pm25,2.000000,µg/m³,Greece,39.366711,22.942922,Good
8,VOLOS-1,pm10,18.000000,µg/m³,Greece,39.366711,22.942922,Good
33,Utrecht NLH,pm10,27.625000,µg/m³,Netherlands,52.094751,5.134536,Good
41,Bangeløkka,pm25,7.079386,µg/m³,Norway,59.733270,10.211670,Good


In [42]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2051 entries, 6 to 7074
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   city           2051 non-null   object 
 1   parameter      2051 non-null   object 
 2   value          2051 non-null   float64
 3   unit           2051 non-null   object 
 4   country        2051 non-null   object 
 5   latitude       2051 non-null   float64
 6   longitude      2051 non-null   float64
 7   quality_label  2051 non-null   object 
dtypes: float64(3), object(5)
memory usage: 144.2+ KB


In [43]:
#filtered_df.to_excel('openaq_data_eu_l.xlsx', index=False)